In [11]:
# ================================================================
# Customer Segmentation with RFM + CLV (Online Retail II Dataset)
# ================================================================

import pandas as pd
import plotly.express as px

# ---------------------------------------------------------------
# Step 1. Load & Clean Dataset
# ---------------------------------------------------------------
file_path = "/content/online_retail_II.csv"   # update with your path
data = pd.read_csv(file_path)

# Convert InvoiceDate to datetime
data["InvoiceDate"] = pd.to_datetime(data["InvoiceDate"])

# Drop missing Customer IDs
data = data.dropna(subset=["Customer ID"])

data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17312 entries, 0 to 23470
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Invoice      17312 non-null  object        
 1   StockCode    17312 non-null  object        
 2   Description  17312 non-null  object        
 3   Quantity     17312 non-null  float64       
 4   InvoiceDate  17312 non-null  datetime64[ns]
 5   Price        17312 non-null  float64       
 6   Customer ID  17312 non-null  float64       
 7   Country      17312 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 1.2+ MB


In [12]:
# Remove cancelled invoices (Invoice starting with 'C')
data = data[~data["Invoice"].astype(str).str.startswith("C")]


In [13]:


# Create TotalPrice = Quantity × Price
data["TotalPrice"] = data["Quantity"] * data["Price"]

# Define snapshot date = one day after the last purchase
snapshot_date = data["InvoiceDate"].max() + pd.Timedelta(days=1)

# ---------------------------------------------------------------
# Step 2. Calculate RFM Metrics
# ---------------------------------------------------------------
# Recency = Days since last purchase
# Frequency = Number of unique invoices
# Monetary = Total spend by the customer

rfm = data.groupby("Customer ID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,
    "Invoice": "nunique",
    "TotalPrice": "sum"
})

In [ ]:
rfm.rename(columns={
    "InvoiceDate": "Recency",
    "Invoice": "Frequency",
    "TotalPrice": "Monetary"
}, inplace=True)

rfm

# ---------------------------------------------------------------
# Step 3. Assign RFM Scores (1–5)
# ---------------------------------------------------------------
# Rule of thumb:
# - Low Recency = better (recent buyers)
# - High Frequency = better
# - High Monetary = better


rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1]) # lower recency = higher score
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 5, labels=[1,2,3,4,5])

rfm

# Combine to get a single score
rfm["RFM_Segment"] = (
    rfm["R_Score"].astype(str) +
    rfm["F_Score"].astype(str) +
    rfm["M_Score"].astype(str)
)

rfm["RFM_Score"] = (
    rfm[["R_Score","F_Score","M_Score"]].astype(int).sum(axis=1)
)

rfm

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score
Customer ID,,,,,,,,
12346.0,326,12,77556.46,2,5,5,255,12
12347.0,2,8,5633.32,5,4,5,545,14
12348.0,75,5,2019.40,3,4,4,344,11
12349.0,19,4,4428.69,5,3,5,535,13
12350.0,310,1,334.40,2,1,2,212,5
...,...,...,...,...,...,...,...,...
18283.0,4,22,2736.65,5,5,4,554,14
18284.0,432,1,461.68,1,2,2,122,5
18285.0,661,1,427.00,1,2,2,122,5


In [ ]:
# ---------------------------------------------------------------
# Step 4. Calculate CLV (Simplified Formula)
# ---------------------------------------------------------------
# CLV = (Average Order Value × Purchase Frequency) × Profit Margin × Time Horizon
# Where:
# - Average Order Value = Monetary / Frequency
# - Purchase Frequency = Frequency (per customer, here we treat raw count)
# - Profit Margin (assumed 10%)
# - Time Horizon (assumed 2 years in dataset)

rfm["AvgOrderValue"] = rfm["Monetary"] / rfm["Frequency"]
profit_margin = 0.1
rfm["CLV"] = (rfm["AvgOrderValue"] * rfm["Frequency"]) * profit_margin * 2

# ---------------------------------------------------------------
# Step 5. Create 4 Segments
# ---------------------------------------------------------------
# We'll combine RFM & CLV to create clear archetypes:
# 🦁 Lion (Champions): High RFM, High CLV
# 🐼 Panda (Loyal Regulars): Medium RFM, Steady CLV
# 🦊 Fox (At Risk): Low Recency but used to spend a lot
# 🐢 Turtle (Lost/Low Value): Low RFM, Low CLV

def pixar_segment(row):
    if row["RFM_Score"] >= 12 and row["CLV"] > rfm["CLV"].median():
        return "🦁 Lion - Champions"
    elif row["RFM_Score"] >= 9 and row["CLV"] <= rfm["CLV"].median():
        return "🐼 Panda - Loyal Regulars"
    elif row["RFM_Score"] < 9 and row["CLV"] > rfm["CLV"].median():
        return "🦊 Fox - At Risk Big Spenders"
    else:
        return "🐢 Turtle - Lost/Low Value"

rfm["PixarSegment"] = rfm.apply(pixar_segment, axis=1)
rfm

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Segment,RFM_Score,AvgOrderValue,CLV,PixarSegment
Customer ID,,,,,,,,,,,
12346.0,326,12,77556.46,2,5,5,255,12,6463.038333,15511.292,🦁 Lion - Champions
12347.0,2,8,5633.32,5,4,5,545,14,704.165000,1126.664,🦁 Lion - Champions
12348.0,75,5,2019.40,3,4,4,344,11,403.880000,403.880,🐢 Turtle - Lost/Low Value
12349.0,19,4,4428.69,5,3,5,535,13,1107.172500,885.738,🦁 Lion - Champions
12350.0,310,1,334.40,2,1,2,212,5,334.400000,66.880,🐢 Turtle - Lost/Low Value
...,...,...,...,...,...,...,...,...,...,...,...
18283.0,4,22,2736.65,5,5,4,554,14,124.393182,547.330,🦁 Lion - Champions
18284.0,432,1,461.68,1,2,2,122,5,461.680000,92.336,🐢 Turtle - Lost/Low Value
18285.0,661,1,427.00,1,2,2,122,5,427.000000,85.400,🐢 Turtle - Lost/Low Value


In [14]:
# ---------------------------------------------------------------
# Step 6. Interactive Visualizations (Bigger Size)
# ---------------------------------------------------------------

# Reset index so Customer ID becomes a column
rfm_reset = rfm.reset_index()

# Scatter Plot: Recency vs Frequency with Customer ID in hover
fig1 = px.scatter(
    rfm_reset, x="Recency", y="Frequency",
    size="Monetary", color="PixarSegment",
    hover_data=["Customer ID","RFM_Score","CLV"],
    title="Customer Segmentation: Recency vs Frequency (Bubble = Monetary)",
    width=1000, height=700
)
fig1.show()

# Bar Plot: Top 10 CLV Customers
top_clv = rfm.sort_values("CLV", ascending=False).head(100)
fig2 = px.bar(
    top_clv, x=top_clv.index.astype(str), y="CLV",
    color="PixarSegment", text="CLV",
    title="Top 10 Customers by CLV",
    width=1000, height=700
)
fig2.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig2.show()

# Pie Chart: Pixar Segment Distribution
fig3 = px.pie(
    rfm, names="PixarSegment", hole=0.3,
    title="Distribution of Customer Segments",
    width=800, height=600
)
fig3.show()

# ---------------------------------------------------------------
# Step 7. Lookup Function for Any Customer
# ---------------------------------------------------------------
def check_customer_segment(customer_id):
    """Return RFM metrics, CLV, and Pixar Segment for a given customer ID."""
    if customer_id not in rfm.index:
        return f"Customer {customer_id} not found."
    row = rfm.loc[customer_id]
    return {
        "Recency (days)": row["Recency"],
        "Frequency (#orders)": row["Frequency"],
        "Monetary (£)": round(row["Monetary"],2),
        "RFM_Score": row["RFM_Score"],
        "CLV (£)": round(row["CLV"],2),
        "Pixar Segment": row["PixarSegment"]
    }

# Example:
print(check_customer_segment(13085))  # replace with any valid customer ID

ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Customer ID', 'InvoiceDate', 'Invoice', 'TotalPrice'] but received: Recency

Zomato-Style Strategies (Segment by Segment)
Lion – Champions (High RFM, High CLV)
Zomato equivalent: Daily orderers, Zomato Gold/Pro Plus subscribers.
Strategy:
Exclusive memberships (Zomato Gold → free delivery, priority support).
Priority access during high-demand (e.g., IPL match nights).
Gamification badges (“Top 1% foodie in your city”).
Why: They already love the platform — don’t train them to expect discounts.

Panda – Loyal Regulars (Mid CLV, medium RFM)
Zomato equivalent: Weekly orderers, usually same restaurants.
Strategy:
Personalized nudges: “Your favorite biryani place is trending!”
Bundle deals: Combo meals, free delivery on orders above ₹300.
Subscriptions: Push them toward Zomato Gold Lite or credit wallet.
Why: Keeps them consistent while nudging them to spend more per order.

Fox – At Risk Big Spenders (High past CLV, Low Recency)
Zomato equivalent: Used to order a lot but stopped (maybe switched to Swiggy).
Strategy:
Win-back campaigns: “We miss you, here’s free delivery on your next 2 orders.”
Event-based offers: “IPL is back — enjoy pizza with 20% off today only.”
Localized push: Remind them of restaurants they used to order from.
Discount style: 25–40% targeted, short-term coupons (not blanket).
Why: Re-activating a churned customer is worth a tactical discount, but avoid conditioning them to wait for 80% off.

Turtle – Lost / Low Value
Zomato equivalent: Ordered once 2 years ago during Diwali sale, never came back.
Strategy:
Mass campaigns (festive sales, app notifications).
Partner offers with Paytm, banks, or credit cards.
Cross-selling other verticals (Zomato Grocery, Hyperpure, dining-out offers).
Why: Don’t spend personalized money on them — use general marketing blasts.